# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных
 - children — количество детей в семье
 - days_employed — общий трудовой стаж в днях
 - dob_years — возраст клиента в годах
 - education — уровень образования клиента
 - education_id — идентификатор уровня образования
 - family_status — семейное положение
 - family_status_id — идентификатор семейного положения
 - gender — пол клиента
 - income_type — тип занятости
 - debt — имел ли задолженность по возврату кредитов
 - total_income — ежемесячный доход
 - purpose — цель получения кредита

## Откроем файл с данными и изучим общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В данной таблице мы видим, что данные заполнены, есть некоторые неточности, ошибки в заполнении, конкретно в столбце "days_employed" данные не корректны, ведь стаж не может быть отрицательным.

Проверим к примеру первыу строку и допустим, что здесь ошибочный только знак "-",опустим его: 8473/365 = 23 года, тоесть 42 -23 = 19 лет, что вполне допустимо. Но возьмем строку под индексом 4: 340266 / 365 = 932 года при своих 53. Видмо человек работает уже 9 жизней вподряд и судя по всему без выходных.



**Вывод** Видимо придется исключить из теории зависимость от стажа.

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Столбцы days_employed и total_income заполнены не полностью, есть вероятность взаимосвязи между общим трудовым стажем и ежемесячным доходом некоторых дюдей. Возможно кредит пытаются взяь люди не имеющие работы.

## Предобработка данных

### Обработка пропусков

Посмотрим сколько пропусков в датафрейме

In [3]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Мы видим одинаковое количество пропусков в 2х столбцах выведем часть их на экран

In [4]:
data[data['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Проверим есть ли в столбце dob_years значения меньше 18. Ведь мы знаем, что кредит моложе 18 лет не выдают. и посмотрим на них

In [5]:
data[data['dob_years']< 18]['dob_years'].count()

101

In [6]:
data[data['dob_years']< 18].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль


Проверим все ли значения в столбце равны 0.

In [7]:
data[data['dob_years'] == 0]['dob_years'].count()

101

Все верно значение количество значений  меньше 18 равно количеству значений 0 в столбце. Заменим 0 значения медианному значению столбца income_type для каждой группы

In [8]:
for row in data['income_type'].unique():

    data.loc[(data['dob_years'] == 0)&(data['income_type'] == row), 'dob_years'] = data.loc[data['income_type'] ==
                                                                                            row, 'dob_years'].median()

Проверим пропали ли 0 значения

In [9]:
data[data['dob_years'] == 0]['dob_years'].count()

0

Обнаруженые пропуски в столбцах days_employed и total_income могли возникнут в результате не заполнения самих людей подающих заявки на кредит либо в виду программной ошибки. Так как данные в столбце days_employed не корректны, а новые данные взять негде, то попробуем заменить их на боллее правильные, расчитаем стаж сами. В среднем человек без высшего образования начинает работать в 18 лет(не будем учитывать срок службы в армии для мужчин), а с высшим образованием примерно в 23 года. С неоконченным высшим конечно сложнее но мы возьмем средннее 3 года учебы, тоесть отчет начнется с 21 года. Со средним образованием так же возьмем 21 год. С ученой степенью возьмем как и с высшим образованием т.к. в основном в это время уже все работают.

In [10]:
#Определяем стаж работы. Выполняя следующие условия:
#Если у человека неоконченное высшее или среднее тогда из количества его лет вычитается 21 год.
#Если у человека высшее образование или ученая степень тогда из количества его лет вычитается 23 года.
#Если у человека начальное образование тогда вычитается 18 лет.
data.loc[(data['education_id'] == 1)|(data['education_id'] == 2), 'days_employed'] = data['dob_years']-21
data.loc[(data['education_id'] == 0)|(data['education_id'] == 4), 'days_employed'] = data['dob_years']-23
data.loc[data['education_id'] == 3, 'days_employed'] = data['dob_years']-18

проверим остались ли отрицательные значения в столбце days_employed

In [11]:
data[data['days_employed'] <0]['days_employed'].count()

133

Выведем их на экран

In [12]:
data[data['days_employed'] <0].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
23,0,-2.0,21.0,высшее,0,гражданский брак,1,M,сотрудник,0,128265.720871,сыграть свадьбу
42,0,-1.0,20.0,неоконченное высшее,2,Не женат / не замужем,4,F,сотрудник,0,82065.089021,получение образования
153,1,-1.0,20.0,среднее,1,женат / замужем,0,M,компаньон,1,134750.749515,жилье
216,0,-1.0,22.0,высшее,0,гражданский брак,1,F,сотрудник,0,118757.770135,образование
317,0,-2.0,21.0,высшее,0,Не женат / не замужем,4,M,сотрудник,0,NaN,сделка с автомобилем
437,1,-1.0,22.0,высшее,0,женат / замужем,0,M,сотрудник,1,168110.287253,покупка жилой недвижимости
766,0,-2.0,19.0,неоконченное высшее,2,женат / замужем,0,F,сотрудник,0,80812.147802,покупка жилья для сдачи
1277,0,-1.0,20.0,среднее,1,Не женат / не замужем,4,M,сотрудник,0,92387.574189,жилье
1614,0,-1.0,20.0,среднее,1,женат / замужем,0,F,компаньон,0,201818.035916,ремонт жилью
1621,0,-1.0,22.0,высшее,0,Не женат / не замужем,4,M,компаньон,0,289344.998213,операции со своей недвижимостью


Посмотрим какие значения встречаются в столбце days_employed

In [13]:
data[data['days_employed'] <0]['days_employed'].unique()

array([-2., -1., -3.])

То есть это все те люди которые не подпадают под правило выше указанной функции, для нашего исследования это не так важно, т.к. эти люди будут объеденнены в одну группу в итоге, поэтому просто уберем знак "-" 

In [14]:
data['days_employed']=data['days_employed'].abs()

In [15]:
data[data['days_employed'] <0]['days_employed'].unique()

array([], dtype=float64)

От знака минус избавились.

Посмотрим на значения в столбце income_type.

In [16]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Мы видим, что в столбце income_type все типы занятости делятся на 8 категорий, логично предположить что для заполнения пропусков в столбце total_income необходимо сгруппировать данные по этип типам занятости и заменить пропуски на медианное значение этой группы.

In [17]:
for row in data['income_type'].unique():
    
    data.loc[(data['total_income'].isna())&(data['income_type'] ==row), 'total_income'] = data.loc[data['income_type'] == row, 'total_income'].median()

In [18]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**: Мы обнаружили количественные пропуски в столбцах "days_employed" и "total_income". Данные пропуски скорее всего возникли из за программной ошибки. Так же заметили, что в столбце dob_years некоторые значения были равны 0, мы их заменили.

### Замена типа данных

Для начала напишем функцию для объединения людей в группы по возрасту и сохраним в столбец age_group

In [19]:
def age_group(age):
    """
    Возвращает возврастную группу по значению возраста age, используя правила:
    - 'молодежь', если 18 <= age <= 30 лет;
    - 'взрослые', если age более 31 и менее 64, включая 31 и 64;
    - 'пенсионеры' — во всех остальных случаях.
    """
    
    if 18 <= age <= 30:
        return 'молодежь'
    elif 31 <= age <= 64:
        return 'зрелые'
    else:
        return 'пенсионеры' 

</div>
<div class="alert alert-info"> 
<b>Комментарий ревьюера</b> 
    
Такую же функцию можно написать для разбивки на группы по доходу.


</div>

In [20]:
data['age_group'] = data['dob_years'].apply(age_group)

In [21]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,19.0,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,зрелые
1,1,15.0,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,зрелые
2,0,12.0,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,зрелые
3,3,11.0,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,зрелые
4,0,32.0,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,зрелые


В столбцах days_employed и total_income используется вещественный тип данных float64, что не совсем удобно для нас. Переведем их в целое число методом astype.

In [22]:
data['days_employed'] = data['days_employed'].astype('int')

In [23]:
data['dob_years'] = data['dob_years'].astype('int')

In [24]:
data['total_income'] = data['total_income'].astype('int')

Удостоверимся что тип данных изменился

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
age_group           21525 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


посмотрим на уникальные значения столбца количества детей.

In [26]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

мы видим не совсем корректные дынные, а именно -1 и 20. Проверим количество семей с 20 детьми.

In [27]:
data[data['children'] == 20]['children'].count()

76

Как мы видим целых 76 семей. Но если проверить информацию через всем известный поисковик, то таких семей всего лишь 1 на всю страну. Делаем выводы, что это выброс и просто его подправим, уберем 0, скорее всего это просто опечатка. А так же уберем -1 ведь отрицательного значения детей не бывает, они или есть или нет.

In [28]:
data['children'] = data['children'].abs()

In [29]:
data.loc[(data['children'] == 20), 'children'] = 2

Проверим действительно ли изменились значения в столбце

In [ ]:
data['children'].unique()

**Вывод**: Тип данных заменили на целочисленный, в таком виде данные выглядят куда удобней. Так же был обнаружен вброс данных в столбце с количеством детей, данные исправили

### Обработка дубликатов

Выполним поиск полных дубликатов во всем дата фрейме:

In [31]:
data.duplicated().sum()

54

На данный момент мы видим 54 дубликата, но если внимательно посмотрим на вызов таблицы в начале нашего исследования, то увидим, что в некоторых строках есть значения данные которых записаны не в одном регистре.

Приведем все записи столбцов education, family_status, income_type, purpose к нижнему регистру.

In [33]:
data['education'] = data['education'].str.lower()

In [34]:
data['family_status'] = data['family_status'].str.lower()

In [35]:
data['income_type'] = data['income_type'].str.lower()

In [36]:
data['purpose'] = data['purpose'].str.lower()

In [37]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,19,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,зрелые
1,1,15,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,зрелые
2,0,12,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,зрелые
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,зрелые
4,0,32,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,зрелые


Посчитаем еще раз количество полных дубликатов:

In [38]:
data.duplicated().sum()

71

Теперь мы видим, что количество дубликатов изменилось, значит слова в которых был использован верхний регистр к дубликатам не относятся, удалим их и обновим индексы.

In [39]:
data = data.drop_duplicates().reset_index(drop=True)

Убедимся, что сработало.

In [40]:
data.duplicated().sum()

0

**Вывод**: Мы определили дубликаты, привели все строки к единому виду и удалили дубликаты. Данные дубликаты могли появится в связи как с человеческойй ошибкой так и программной. Теперь можем приступать к следующему пункту нашего исследования.

### Лемматизация

Посмотрим внимательно на столбец purpose.

In [41]:
data['purpose'].sort_values().unique()

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

Мы видим много разных запросов которые содержат одинаковый смысл. Чтобы разделить запросы для дальнейшего использования, проведем лемматизацию.

In [42]:
a = [i for i in data['purpose']]
text = ', '.join(a)

In [43]:
lemmas = m.lemmatize(text)

Посчитаем количество часто используемых слов в лемматизированном словаре

In [44]:
Counter(lemmas)

Counter({'покупка': 5897,
         ' ': 33570,
         'жилье': 4460,
         ', ': 21453,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 853,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1314,
         'высокий': 1374,
         'подержанный': 111,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607,
         '\n': 1})

И создадим список на основе часто встречаемых запросов

In [45]:
list_of_requests=['ремонт','жилье','автомобиль','образование','свадьба','операция','недвижимость','строительство']

Напишем функцию которая будет принимать на вход значение столбца purpose и возвращает в новый столбец уже его лемматизированное слово

In [46]:
def query_reguest(query):
    lemma = m.lemmatize(query)
    for i in lemma:
        if i in list_of_requests:
            
            try:
                return i
            except:
        
                return print(f'слова {i} нет в списке list_of_requests')
    

In [47]:
data['requests'] = data['purpose'].apply(query_reguest)

Выведем таблицу на экран и посмотрим уникальные значения нового столбца

In [48]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,requests
0,1,19,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,зрелые,жилье
1,1,15,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,зрелые,автомобиль
2,0,12,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,зрелые,жилье
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,зрелые,образование
4,0,32,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,зрелые,свадьба


In [49]:
data['requests'].unique()

array(['жилье', 'автомобиль', 'образование', 'свадьба', 'операция',
       'недвижимость', 'строительство', 'ремонт'], dtype=object)

**Вывод**: Мы получили слова приведенные к единственной форме и убрали лишние запросы чтобы в дальнейшем их можно было группировать по запросам.

### Категоризация данных

Объеденим необходимые нам данные в таблицу group_filter

In [50]:
group_filter = data[['children','family_status','family_status_id','total_income','income_type','age_group','requests','education','debt']]

И выведем на экран

In [51]:
group_filter.head(10)

,children,family_status,family_status_id,total_income,income_type,age_group,requests,education,debt
0,1,женат / замужем,0,253875,сотрудник,зрелые,жилье,высшее,0
1,1,женат / замужем,0,112080,сотрудник,зрелые,автомобиль,среднее,0
2,0,женат / замужем,0,145885,сотрудник,зрелые,жилье,среднее,0
3,3,женат / замужем,0,267628,сотрудник,зрелые,образование,среднее,0
4,0,гражданский брак,1,158616,пенсионер,зрелые,свадьба,среднее,0
5,0,гражданский брак,1,255763,компаньон,молодежь,жилье,высшее,0
6,0,женат / замужем,0,240525,компаньон,зрелые,операция,высшее,0
7,0,женат / замужем,0,135823,сотрудник,зрелые,образование,среднее,0
8,2,гражданский брак,1,95856,сотрудник,зрелые,свадьба,высшее,0
9,0,женат / замужем,0,144425,сотрудник,зрелые,жилье,среднее,0


**Вывод**: Мы создали новую таблицу по параметрам: количество детей, id количества детей, уровень зарплаты, тип рабочего, возрастная группа цель кредита, образование и  количество просрочек так как именно эти данные нужны нам для анализа

## Ответим на вопросы:

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Сгруппируем данные по количеству детей и количеству просрочек.

In [52]:
data_grouped = group_filter.groupby(['children']).agg({ 'children':'count','debt':'sum'})

Создадим столбец в котором посчитаем процент невозврата кредита в зависимости от общего количества детей.

In [53]:
data_grouped

,children,debt
children,,
0,14091,1063
1,4855,445
2,2128,202
3,330,27
4,41,4
5,9,0


In [54]:
data_grouped['all_results']=data_grouped['debt']/data_grouped['children'] *100

In [55]:
data_grouped.sort_values('all_results',ascending=False)

,children,debt,all_results
children,,,
4,41,4,9.756098
2,2128,202,9.492481
1,4855,445,9.165808
3,330,27,8.181818
0,14091,1063,7.543822
5,9,0,0.000000


И так, мы видим, что с увеличением количества детей в семье растет и количество просрочек по кредиту. Добавим в таблицу возраст людей и посмотрим на результат

In [56]:
data_grouped = group_filter.groupby(['age_group','children']).agg({ 'children':'count','debt':'sum'})

In [57]:
data_grouped['all_results']=data_grouped['debt']/data_grouped['children'] *100

In [58]:
data_grouped

children  debt  all_results
age_group  children                             
зрелые     0            11090   782     7.051398
           1             3754   329     8.763985
           2             1674   151     9.020311
           3              280    24     8.571429
           4               35     3     8.571429
           5                9     0     0.000000
молодежь   0             2147   234    10.898929
           1             1061   114    10.744581
           2              453    51    11.258278
           3               50     3     6.000000
           4                6     1    16.666667
пенсионеры 0              854    47     5.503513
           1               40     2     5.000000
           2                1     0     0.000000

**Вывод**: И так, мы видим, что у зрелых людей больше всего просрочек у семей с 2 детьми, дальше примерно одинаково. У молодежи больше всего просрочек у людей с 4 детьми, но так как всего таких семей 6, то это очень маленькое количество для проведения анализа.Дальше идут с 2, а вот с 1 и без детей примерно на одном уровне. У пенсионеров количество просрочек больше всего с 1 ребенком и без детей примерно на одном уровне

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Выведем на экран таблицу зависимости семейного положения и просроченных платежей

In [59]:
data_grouped_family = group_filter.groupby(['family_status']).agg({ 'family_status_id':'count','debt':'sum'})

In [60]:
data_grouped_family['percent'] = data_grouped_family['debt'] / data_grouped_family['family_status_id'] * 100

In [61]:
data_grouped_family.sort_values('percent', ascending=False)

,family_status_id,debt,percent
family_status,,,
не женат / не замужем,2810,274,9.750890
гражданский брак,4151,388,9.347145
женат / замужем,12339,931,7.545182
в разводе,1195,85,7.112971
вдовец / вдова,959,63,6.569343


Мы видим, что человек потерявший супруга/супругу более серьездно относится к выплате кредита, чем холостые люди. Попробуем капнуть не много глубже и разделим так же людей на возрастные категории.

In [62]:
data_grouped_family_percent = group_filter.groupby(['age_group','family_status']).agg({ 'family_status_id':'count','debt':'sum'})

И так же найдем процент по не возврату кредитов для каждой категории.

In [63]:
data_grouped_family_percent['percent'] = data_grouped_family_percent['debt'] / data_grouped_family_percent['family_status_id'] *100

In [64]:
data_grouped_family_percent

family_status_id  debt    percent
age_group  family_status                                           
зрелые     в разводе                          1029    63   6.122449
           вдовец / вдова                      783    50   6.385696
           гражданский брак                   3226   296   9.175449
           женат / замужем                   10094   722   7.152764
           не женат / не замужем              1710   158   9.239766
молодежь   в разводе                           117    17  14.529915
           вдовец / вдова                        7     1  14.285714
           гражданский брак                    785    84  10.700637
           женат / замужем                    1794   187  10.423634
           не женат / не замужем              1014   114  11.242604
пенсионеры в разводе                            49     5  10.204082
           вдовец / вдова                      169    12   7.100592
           гражданский брак                    140     8   5.714286
           женат / замужем                     451    22   4.878049
           не женат / не замужем                86     2   2.325581

**Вывод**: И так: Исходя из таблицы мы видим, что к примеру зрелые люди чаще всего имеют просрочки по кредитам не женатые и в гражданском браке скорее всего это те люди которые живут что называется сегодняшним днем и не думают о последствиях.В то время как в разводе, замужние или вдовцы имеют более низкий процент по невыплатам в срок. У молодежи процент просрочек гораздо выше по всем пунктам, но особенно выделяются в разводе и вдовцы, хоть конечно их количество гораздо меньше чем к примеру у не женатых. У пенсионеров лидируют по невыплатам люди в разводе и вдовцы хотя общее количество пенсионеров меньше чем людей из других групп они более качественно осуществляют платеди по кредитам.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [65]:
data_grouped_family_income = group_filter.groupby(['age_group']).agg({ 'total_income':'sum','debt':'sum'})

In [66]:
data_grouped_family_income['percent'] = data_grouped_family_income['debt'] / data_grouped_family_income['total_income'] * 100

In [67]:
data_grouped_family_income

,total_income,debt,percent
age_group,,,
зрелые,2831027277,1289,0.000046
молодежь,596098608,403,0.000068
пенсионеры,119640219,49,0.000041


Мы видим, что процент просроченных платежей очень маленький хотя с увеличением заработной платы количество просроченных платежей растет. Попробуем посчитать количество людей в каждой возрастной группе и выясним отношение просроченных платежей к количеству людей в каждой возрастной группе.

In [68]:
data_grouped_family_income = group_filter.groupby(['age_group']).agg({ 'total_income':'sum','debt':'sum','income_type':'count'})

In [69]:
data_grouped_family_income['percent'] = data_grouped_family_income['debt'] / data_grouped_family_income['income_type'] * 100

In [70]:
data_grouped_family_income

,total_income,debt,income_type,percent
age_group,,,,
зрелые,2831027277,1289,16842,7.653485
молодежь,596098608,403,3717,10.842077
пенсионеры,119640219,49,895,5.474860


**Вывод**: Мы видим что не смотря на большую заработную плату у зрелых людей процент просроченных платежей находится на 2м месте, на третьем пенсионеры, ну а лидирует молодежь. Видимо чем моложе люди, тем они менее ответственнее.

- Как разные цели кредита влияют на его возврат в срок?

Сгруппируем таблицу по запросам и посчитаем общее количество запросов и сумму просроченных платежей, так же добавим столбец процентов просроченных платежей и отсортируем по убыванию

In [71]:
data_grouped_family_request = group_filter.groupby(['requests']).agg({'total_income':'count','debt':'sum'})

In [72]:
data_grouped_family_request['percent'] = data_grouped_family_request['debt'] / data_grouped_family_request['total_income'] * 100

In [73]:
data_grouped_family_request.sort_values('percent', ascending=False)

,total_income,debt,percent
requests,,,
автомобиль,4306,403,9.359034
образование,4013,370,9.220035
свадьба,2324,186,8.003442
операция,2604,205,7.872504
строительство,1878,144,7.667732
жилье,3201,225,7.029053
недвижимость,2521,173,6.862356
ремонт,607,35,5.766063


Как мы видим на первые по количеству просроченных платежей запросы на автомобиль, образование и свадьбу. Совместим это с возрастом и проверим как это отразится на результате и так же добавим колонку с процентами просроченных платажей.

In [74]:
data_all_group = group_filter.groupby(['age_group','requests']).agg({'education':'count','debt':'sum'})

In [75]:
data_all_group['percent'] = data_all_group['debt'] / data_all_group['education'] * 100

In [76]:
data_all_group

education  debt    percent
age_group  requests                                 
зрелые     автомобиль          3434   309   8.998253
           жилье               2514   164   6.523469
           недвижимость        1973   125   6.335530
           образование         3120   268   8.589744
           операция            2035   151   7.420147
           ремонт               450    22   4.888889
           свадьба             1833   142   7.746863
           строительство       1483   108   7.282535
молодежь   автомобиль           686    84  12.244898
           жилье                558    54   9.677419
           недвижимость         440    41   9.318182
           образование          719    93  12.934631
           операция             460    48  10.434783
           ремонт               131    11   8.396947
           свадьба              392    38   9.693878
           строительство        331    34  10.271903
пенсионеры автомобиль           186    10   5.376344
           жилье                129     7   5.426357
           недвижимость         108     7   6.481481
           образование          174     9   5.172414
           операция             109     6   5.504587
           ремонт                26     2   7.692308
           свадьба               99     6   6.060606
           строительство         64     2   3.125000

**Вывод**: Как мы видим лидирует по количеству просроченных платежей по прежнему молодежь при том, что в числе самых поулярных запросов как по количеству просроченных платежей так и по общему количеству находится автомобиль. Дальше идет образование и операция у других же категорий ситуация не много лучше, у зрелых людей идет так же автомобиль и образование. А вот у пенсионеров ситуация гораздо лучше они гораздо чаще выплачивают кредит вовремя.

- Как образование влияет на возврат кредита в срок?

Сгруппируем данные по столбцам возраст и образование, добаивм столбец с процентами и посмотрим на него

In [77]:
data_education = group_filter.groupby(['age_group','education']).agg({'education':'count','debt':'sum'})

In [78]:
data_education

education  debt
age_group  education                           
зрелые     высшее                    3921   191
           начальное                  219    26
           неоконченное высшее        398    38
           среднее                  12299  1034
           ученая степень               5     0
молодежь   высшее                    1185    78
           начальное                   30     5
           неоконченное высшее        331    30
           среднее                   2171   290
пенсионеры высшее                     144     9
           начальное                   33     0
           неоконченное высшее         15     0
           среднее                    702    40
           ученая степень               1     0

In [79]:
data_education['percent'] = data_education['debt'] / data_education['education'] * 100

In [80]:
data_education

education  debt    percent
age_group  education                                      
зрелые     высшее                    3921   191   4.871206
           начальное                  219    26  11.872146
           неоконченное высшее        398    38   9.547739
           среднее                  12299  1034   8.407188
           ученая степень               5     0   0.000000
молодежь   высшее                    1185    78   6.582278
           начальное                   30     5  16.666667
           неоконченное высшее        331    30   9.063444
           среднее                   2171   290  13.357900
пенсионеры высшее                     144     9   6.250000
           начальное                   33     0   0.000000
           неоконченное высшее         15     0   0.000000
           среднее                    702    40   5.698006
           ученая степень               1     0   0.000000

**Вывод**: Мы видим, что на первом месте по прежнему молодежь с начальным и средним образованием дальше идут категория зрелых людей с начальным образовнием и зрелые с неоконченным высшим.

## Общий вывод

Мы провели большое исследование зависимости просроченных платежей от разных факторов. Преобразовали пропуски, удалили дубликаты. Воспользовались лемматизацией чтобы выявить основные виды запросов. В итоге выяснили, что самая не платежеспособная группа людей - это люди от 18 до 30 лет в разводе либо не женаты/не замужем покупающие автомобиль либо оплачивающие образование со средним или начальным образованием. Видимо люди образование играет далеко не последнюю роль правильной оценки ситуации при выборе кредита, к тому же в таком возрасте не совсем задумываются о последствиях, что конечно чревато в итоге как для банка, так и для самих заемщиков. Так же и это относится к людям в возрасте от 31 до 64 лет с начальным образованием, но здсь уже процент по просроченным кредитам меньше. Скорее всего если эту группу людей разделить еще на подгруппу, то ситуация окажется такой, что чем старше человек, тем ответственне он относится к свои обязанностям. У людей старше 65 лет процент по просрочкам всех платежей гораздо меньше исключение составляет категория людей которые в разводе или уже вдовцы. Размер оплаты труда особую роль в качестве платежей по кредитам не играет.